In [81]:
import numpy as np
from numpy.linalg import norm
import pandas as pd
import itertools
import random
from matplotlib import pyplot as plt
from pair_trading_foundations.data_generation import ExecutePairTrading, generate_training_data
random.seed(23)
import pstats
from tqdm.notebook import tqdm
tqdm.pandas()

In [82]:
data = pd.read_csv('Data/sp500_full_20181231_to_20231229.csv')

In [83]:
set(data['GICS Sector'])

{'Communication Services',
 'Consumer Discretionary',
 'Consumer Staples',
 'Energy',
 'Financials',
 'Health Care',
 'Industrials',
 'Information Technology',
 'Materials',
 'Real Estate',
 'Utilities'}

# Draft below

## Get Combinations

In [84]:
tickers = list(set(data.Ticker))
combinations = list(itertools.combinations(tickers, 2))

In [85]:
len(combinations)

125250

# Initiate data tables to store the generated results

In [86]:
recorded_info_tb = pd.DataFrame(columns=[
    'ticker1', 
    'ticker2',
    'target_date',
    'spread',
    'spread_mean',
    'spread_std',
    'abs_spread_mean_l28',
    'abs_spread_std_l28',
    'spread_normed'
])

features_tb = pd.DataFrame(columns=[
    'ticker1', 
    'ticker2',
    'target_date',
    'same_sector_flag',
    'same_sub_industry_flag',
    'cos_sim',
    'corr_coef',
    'abs_spread_normed_max',
    'abs_spread_normed_90th',
    'abs_spread_normed_75th',
    'abs_spread_normed_median',
    'abs_spread_normed_l7_avg',
    'abs_spread_normed_l14_avg'
])

labels_tb = pd.DataFrame(columns=[
    'total_pnl',
    'total_pnl_l28_mean_std'
])

## Generate

In [87]:
data.head()

,Date,Open,High,Low,Close,Adj Close,Volume,Ticker,GICS Sector,GICS Sub-Industry
0,2018-12-31,190.339996,191.649994,188.500000,190.539993,155.818451,1804400.0,MMM,Industrials,Industrial Conglomerates
1,2019-01-02,187.820007,190.990005,186.699997,190.949997,156.153717,2475200.0,MMM,Industrials,Industrial Conglomerates
2,2019-01-03,188.279999,188.279999,182.889999,183.759995,150.273972,3358200.0,MMM,Industrials,Industrial Conglomerates
3,2019-01-04,186.750000,191.979996,186.029999,191.320007,156.456284,2995100.0,MMM,Industrials,Industrial Conglomerates
4,2019-01-07,191.360001,192.300003,188.660004,190.880005,156.096466,2162200.0,MMM,Industrials,Industrial Conglomerates


In [145]:
ticker1 = 'META'
ticker2 = 'WAT'

In [146]:
# Get a list of unique dates for later use
all_dates = data['Date'].unique()

In [147]:
# Flag indicating whether the two tickers are from the same sector
same_sector_flag = data[data.Ticker==ticker1]['GICS Sector'].values[0] == data[data.Ticker==ticker2]['GICS Sector'].values[0]
same_sub_industry_flag = data[data.Ticker==ticker1]['GICS Sub-Industry'].values[0] == data[data.Ticker==ticker2]['GICS Sub-Industry'].values[0]

In [148]:
vec1_full = data[['Ticker','Date','Close']][data.Ticker==ticker1].reset_index(drop=True)
vec2_full = data[['Ticker','Date','Close']][data.Ticker==ticker2].reset_index(drop=True)

In [149]:
# # Number of days in the data
# num_days_total = len(vec1_full)

In [150]:
# # Keep 500 days for training and 120 days for label calculation
# possible_indices_to_sample = list(range(500, num_days_total-119))

In [151]:
# sampled_indices = random.choices(possible_indices_to_sample, k=100)

In [152]:
lj_df = pd.merge(vec1_full,vec2_full,on='Date',how='left',suffixes=['_P1','_P2'])
lj_df.head()

,Ticker_P1,Date,Close_P1,Ticker_P2,Close_P2
0,META,2018-12-31,131.089996,WAT,188.649994
1,META,2019-01-02,135.679993,WAT,183.410004
2,META,2019-01-03,131.740005,WAT,176.339996
3,META,2019-01-04,137.949997,WAT,179.779999
4,META,2019-01-07,138.050003,WAT,187.240005


In [153]:
# # Absolute value of the difference of the two stocks
# abs_spread = abs(vec1_sub1 - vec2_sub1)
# abs_spread_mean = np.mean(abs_spread)
# abs_spread_std = np.std(abs_spread)

# # Sometimes, historical data might be too strict to get a signal for trade in
# abs_spread_mean_l28 = np.mean(abs_spread[-28:])
# abs_spread_std_l28 = np.std(abs_spread[-28:])

lj_df['abs_spread'] = (lj_df['Close_P1'] - lj_df['Close_P2']).abs()
lj_df['abs_spread_mean'] = lj_df.rolling(500).abs_spread.mean()
lj_df['abs_spread_std'] = lj_df.rolling(500).abs_spread.std()
lj_df['abs_spread_mean_l28'] = lj_df.rolling(28).abs_spread.mean()
lj_df['abs_spread_std_l28'] = lj_df.rolling(28).abs_spread.std()
lj_df['spread_normed'] = (lj_df['abs_spread']-lj_df['abs_spread_mean'])/lj_df['abs_spread_std']
lj_df['abs_spread_normed_max'] = lj_df.spread_normed.abs().rolling(500).max()
lj_df['abs_spread_normed_90th'] = lj_df.spread_normed.abs().rolling(500).quantile(0.9)
lj_df['abs_spread_normed_75th'] = lj_df.spread_normed.abs().rolling(500).quantile(0.75)
lj_df['abs_spread_normed_median'] = lj_df.spread_normed.abs().rolling(500).median()
lj_df['abs_spread_normed_l7_avg'] = lj_df.spread_normed.abs().rolling(7).mean()
lj_df['abs_spread_normed_l14_avg'] = lj_df.spread_normed.abs().rolling(14).mean()
lj_df.tail()


# abs_spread_normed_max = max(abs(spread_normed))
# abs_spread_normed_90th = np.percentile(abs(spread_normed),90)
# abs_spread_normed_75th = np.percentile(abs(spread_normed),75)
# abs_spread_normed_median = np.percentile(abs(spread_normed),50)

# # latest 7 day/14 day avg normalized spread
# ## These could help predict whether a trading signal will appear
# abs_spread_normed_l7_avg = abs(np.mean(spread_normed[-7:]))
# abs_spread_normed_l14_avg = abs(np.mean(spread_normed[-14:]))

,Ticker_P1,Date,Close_P1,Ticker_P2,Close_P2,abs_spread,abs_spread_mean,abs_spread_std,abs_spread_mean_l28,abs_spread_std_l28,spread_normed,abs_spread_normed_max,abs_spread_normed_90th,abs_spread_normed_75th,abs_spread_normed_median,abs_spread_normed_l7_avg,abs_spread_normed_l14_avg
1254,META,2023-12-22,353.390015,WAT,329.739990,23.650024,103.764481,66.817611,43.008214,20.422297,-1.199002,3.943149,2.868902,2.276608,1.331487,1.242128,1.157295
1255,META,2023-12-26,354.829987,WAT,331.970001,22.859985,103.754441,66.829412,41.051785,19.588654,-1.210462,3.943149,2.868902,2.276608,1.331487,1.218459,1.169034
1256,META,2023-12-27,357.829987,WAT,333.040009,24.789978,103.748720,66.836062,39.588213,19.200496,-1.181379,3.943149,2.868902,2.276608,1.331487,1.197023,1.172867
1257,META,2023-12-28,358.320007,WAT,332.769989,25.550018,103.727321,66.859426,38.072500,18.539125,-1.169279,3.943149,2.868902,2.276608,1.331487,1.189942,1.181351
1258,META,2023-12-29,353.959991,WAT,329.230011,24.729980,103.724680,66.862525,36.342856,17.366422,-1.181450,3.943149,2.868902,2.276608,1.331487,1.182664,1.198456


In [44]:
# Cosine sim
def cos_sim(row):
    print(row)
    vec1_sub1 = row['Close_P1']
    vec2_sub1 = row['Close_P2']
    return np.dot(vec1_sub1, vec2_sub1) / (norm(vec1_sub1) * norm(vec2_sub1))

# Correlation coef
def corr_coef(vec1_sub1, vec2_sub):
    return np.corrcoef(vec1_sub1, vec2_sub1)[0, 1]

In [60]:
# lj_df['cos_sim'] = 
lj_df.dropna()[['Close_P1','Close_P2']].rolling(500,axis=0).apply(lambda x: x.Close_P1.sum())

/var/folders/f4/28wskrvx4yz7gqswwx15x1bh0000gn/T/ipykernel_16821/45699822.py:2: FutureWarning: The 'axis' keyword in DataFrame.rolling is deprecated and will be removed in a future version. Call the method without the axis keyword instead.
  lj_df.dropna()[['Close_P1','Close_P2']].rolling(500,axis=0).apply(lambda x: x.Close_P1.sum())


,Close_P1,Close_P2
998,NaN,NaN
999,NaN,NaN
1000,NaN,NaN
1001,NaN,NaN
1002,NaN,NaN
...,...,...
1254,NaN,NaN
1255,NaN,NaN
1256,NaN,NaN
1257,NaN,NaN


# Trading scoring

In [154]:
lj_df.head()

,Ticker_P1,Date,Close_P1,Ticker_P2,Close_P2,abs_spread,abs_spread_mean,abs_spread_std,abs_spread_mean_l28,abs_spread_std_l28,spread_normed,abs_spread_normed_max,abs_spread_normed_90th,abs_spread_normed_75th,abs_spread_normed_median,abs_spread_normed_l7_avg,abs_spread_normed_l14_avg
0,META,2018-12-31,131.089996,WAT,188.649994,57.559998,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,META,2019-01-02,135.679993,WAT,183.410004,47.730011,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,META,2019-01-03,131.740005,WAT,176.339996,44.599991,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,META,2019-01-04,137.949997,WAT,179.779999,41.830002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,META,2019-01-07,138.050003,WAT,187.240005,49.190002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [155]:
np.where(lj_df.Date=='2022-11-07')[0][0]

971

In [156]:
# Assuming we are scoring for the pair on 2022-09-23
abs_spread_mean = lj_df['abs_spread_mean'][lj_df.Date=='2022-09-23'].values[0]
abs_spread_std = lj_df['abs_spread_std'][lj_df.Date=='2022-09-23'].values[0]

# look forward 120 days
idx = np.where(lj_df.Date=='2022-09-23')[0][0]
vec1 = lj_df['Close_P1'][(idx+1):(idx+121)].values
vec2 = lj_df['Close_P2'][(idx+1):(idx+121)].values

In [157]:
base_fund = 100
split = 0.5

entry_thresh = abs_spread_mean + 1.5*abs_spread_std
exit_thresh = abs_spread_mean + 0.1*abs_spread_std

In [158]:
# absolute spread of the look forward set
abs_spread = abs(np.array(vec1) - np.array(vec2))

In [159]:
# entry_signals = np.array([0, 1, 1, 1, 0, 0, 1, 1, 0, 0])
# exit_signals =  np.array([1, 0, 0, 0, 1, 0, 0, 0, 0, 1])


# Calculate at which position did the signals appear
entry_signals = np.array([0]+[1 if abs_spread[i-1] >= entry_thresh else 0 for i in range(1, len(abs_spread))])
exit_signals = np.array([0]+[1 if abs_spread[i-1] <= exit_thresh else 0 for i in range(1, len(abs_spread))])

In [160]:
entry_positions = np.where(entry_signals == 1)[0]
exit_positions = np.where(exit_signals == 1)[0]
pairs = []

for entry_pos in entry_positions:
    # Find the first exit position that is greater than the entry position
    next_exit_pos = exit_positions[exit_positions > entry_pos]
    if next_exit_pos.size > 0:
        exit_pos = next_exit_pos[0]
    else:
        # Default exit position if no exit signal is found after the entry signal
        exit_pos = len(entry_signals) - 1
    pairs.append((entry_pos, exit_pos))

In [161]:
# Storing in a temporary table
temp_tb = pd.DataFrame(pairs)
temp_tb.columns = ['entry_idx', 'exit_idx']
temp_tb = temp_tb.groupby('exit_idx').min().reset_index()

In [162]:
temp_tb

,exit_idx,entry_idx
0,119,7


When vectorizing the code above, be mindful of scenarios where there are multiple entry and exits during the 120 day window

In [163]:
# At each position, calculate each stocks relative pct delta compated to l7. Short the one with larger pct delta
def long_stock1_flag(stock1, stock2, idx):
    """
    This function is a ultility function to determine which stock to long/short given an entry signal.
    It:
        1. Takes the prices of two stocks, and the position where the entry signal appears 
        2. Calculate the percentage deltas between the current price and the price 7 days ago (or the earliest record) for each stock

    Then we will tell the ago to short the one with higher percentage delta and long the other.

    The function returns a boolean on whether we should long the stock 1.
    """
    stock1_current = stock1[idx]
    stock1_ref = stock1[max(0, idx-7)]

    stock2_current = stock2[idx]
    stock2_ref = stock2[max(0, idx-7)]

    pct_delta_1 = (stock1_current/stock1_ref) - 1
    pct_delta_2 = (stock2_current/stock2_ref) - 1

    if pct_delta_1 >= pct_delta_2:
        return False
    else:
        return True

In [164]:
# get the price for each stock when the entry and exit signal appears
temp_tb['stock1_price_entry'] = vec1[temp_tb['entry_idx']] 
temp_tb['stock1_price_exit'] = vec1[temp_tb['exit_idx']] 
temp_tb['stock2_price_entry'] = vec2[temp_tb['entry_idx']] 
temp_tb['stock2_price_exit'] = vec2[temp_tb['exit_idx']] 

# calculate whether we should long each stock
temp_tb['long_stock_1'] = [long_stock1_flag(vec1, vec2, x) for x in temp_tb.entry_idx]

temp_tb

,exit_idx,entry_idx,stock1_price_entry,stock1_price_exit,stock2_price_entry,stock2_price_exit,long_stock_1
0,119,7,138.979996,195.610001,290.839996,298.970001,True


In [165]:
pnls = []
for row in range(temp_tb.shape[0]):
    long_pnl=0
    short_pnl=0
    if temp_tb.long_stock_1[row]:
        # calculate pnl when we long stock 1 and short stock 2
        long_pnl = base_fund * split * ((temp_tb.stock1_price_exit.values[row] - temp_tb.stock1_price_entry.values[row])/temp_tb.stock1_price_entry.values[row])
        short_pnl = base_fund * (1-split) * ((temp_tb.stock2_price_entry.values[row] - temp_tb.stock2_price_exit.values[row])/temp_tb.stock2_price_entry.values[row])
    else:
        # calculate pnl when we long stock 2 and short stock 1
        long_pnl = base_fund * (1-split) * ((temp_tb.stock2_price_exit.values[row] - temp_tb.stock2_price_entry.values[row])/temp_tb.stock2_price_entry.values[row])
        short_pnl = base_fund * (split) * ((temp_tb.stock1_price_entry.values[row] - temp_tb.stock1_price_exit.values[row])/temp_tb.stock1_price_entry.values[row])
    pnls.append(long_pnl+short_pnl)
temp_tb['pnl'] = pnls

In [166]:
temp_tb

,exit_idx,entry_idx,stock1_price_entry,stock1_price_exit,stock2_price_entry,stock2_price_exit,long_stock_1,pnl
0,119,7,138.979996,195.610001,290.839996,298.970001,True,18.975761


# Testing the module

In [ ]:
plt.plot(vec1, label='Stock1', color='red')
plt.plot(vec2, label='Stock2', color='green')
plt.plot(abs(vec1 - vec2), label='Absolute Spread')
plt.axvline(x=48, color='red', linestyle='--', label='Vertical Line')
plt.axvline(x=119, color='red', linestyle='--', label='Vertical Line')

In [ ]:
with cProfile.Profile() as pr:
    recorded_info_tb, features_tb, labels_tb = generate_training_data(
        # data=data[data['GICS Sector'].isin(['Information Technology'])],
        data=data,
        training_len=500,
        test_len=120,
        sample_size_per_pair=2
    )

stats = pstats.Stats(pr)
stats.sort_stats(pstats.SortKey.TIME)
# Now you have two options, either print the data or save it as a file
stats.print_stats() # Print The Stats
stats.dump_stats("File/path.prof") # Saves the data in a file, can me used to see the data visually
features_tb
labels_tb

recorded_info_tb.to_csv('Data/all_recorded_info_tb.csv', index=False)
features_tb.to_csv('Data/all_features_tb.csv', index=False)
labels_tb.to_csv('Data/all_labels_tb.csv', index=False)